In [1]:
import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

import pandas as pd
import numpy as np

from config import *
from Data.raw_data_loader import load_raw_crypto_csv
from Models.zscore import zscore_signal
from Models.trend import trend_signal
from Backtest.engine import run_backtest
from Backtest.metrics import build_summary_table

df = load_raw_crypto_csv(DATA_PATH)

df = df[(df["ETH-USD_close"] > 0)].copy()
df = df.loc["2017-11-12":].copy()

eth_ret_raw = df["ETH-USD_close"].pct_change()
df = df.loc[eth_ret_raw.abs() < 1.0].copy()   # remove extreme bad ticks

eth_price = df["ETH-USD_close"].astype(float)

print("Data loaded:", df.shape)


Data loaded: (3008, 12)


In [2]:
eth_ret = eth_price.pct_change().dropna()

print("ETH Annualized Vol:", float(eth_ret.std() * np.sqrt(365)))
print("ETH CAGR:", float((eth_price.iloc[-1] / eth_price.iloc[0]) ** (365 / len(eth_price)) - 1))


ETH Annualized Vol: 0.8880602279145787
ETH CAGR: 0.25991736773019003


In [3]:
z_pos = zscore_signal(
    df,
    price_column="ETH-USD_close",
)

trend_pos = trend_signal(
    df,
    price_column="ETH-USD_close",
)


In [4]:
z_results = run_backtest(
    price_series=eth_price,
    position=z_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    annual_borrow_rate=ANNUAL_BORROW_RATE,
    leverage_cap=LEVERAGE_CAP,
)

trend_results = run_backtest(
    price_series=eth_price,
    position=trend_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    annual_borrow_rate=ANNUAL_BORROW_RATE,
    leverage_cap=LEVERAGE_CAP,
)


In [5]:
summary = build_summary_table({
    "Z-Score": {"returns": z_results["net_returns"], "position": z_pos},
    "Trend": {"returns": trend_results["net_returns"], "position": trend_pos},
})
summary


,Sharpe,Sortino,CAGR,MaxDD,Calmar,AnnualReturn,AnnualVol,Observations,AnnualTurnover
Strategy,,,,,,,,,
Z-Score,0.669990,0.937523,0.129529,-0.430459,0.300909,0.145262,0.216812,3008,5.374616
Trend,0.772334,1.062281,0.325912,-0.918567,0.354804,0.745408,0.965137,3008,2.002161
